In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import pandas as pd

In [4]:
df = pd.read_csv('/home/dmitrii/vscode_projects/PMLDL/Assignment1/data/raw/filtered.tsv', sep='\t', encoding='utf-8')
print(df.shape)
df.sample(10)

(577777, 7)


,Unnamed: 0,reference,translation,similarity,lenght_diff,ref_tox,trn_tox
336880,336880,I'll deal with that little weasel myself.,I'll take care of this guy myself.,0.607365,0.166667,0.923489,0.000165
407228,407228,Don't shoot him.,don't shoot.,0.899540,0.235294,0.803848,0.004890
150143,150143,Knew that you had no chance of killing me.,he knew you had no chance to kill me.,0.908300,0.116279,0.186780,0.816266
409806,409806,And why the fuck isn't this war ending?,and why doesn't the war end?,0.886870,0.275000,0.997234,0.000052
96207,96207,"Pompous sod, aren't you?","you're a pompous bastard, aren't you?",0.798035,0.342105,0.002058,0.999495
328534,328534,Prick.,cock.,0.731666,0.142857,0.002022,0.819907
427601,427601,"This guy's gonna kill again, and we don't know...",he's going to kill again and we don't know any...,0.942406,0.059701,0.982934,0.256473
79341,79341,"if you were taken hostage, you would've fucked...","if he had kept you hostage, you would have com...",0.805749,0.060606,0.995495,0.040181
101726,101726,"You know, it's a damn shame you can't take thi...",it's a hell of a shame you can't accept that.,0.736532,0.132075,0.988483,0.001434
468741,468741,"What is she, a superhero or a softcore porno?","what is he, a superhero or a porn actress?",0.822621,0.065217,0.058013,0.717966


In [ ]:
from datasets import load_metric
import evaluate
from tqdm import trange
from transformers import RobertaForSequenceClassification, RobertaTokenizer, AutoTokenizer
import torch


def predict_toxicity(texts, device='cpu', clf_name = 's-nlp/roberta_toxicity_classifier_v1'):
    """
    This function predict toxicity of the words

    Args:
        texts (_type_): _description_
        device (str, optional): _description_. Defaults to 'cpu'.
        clf_name (str, optional): _description_. Defaults to 's-nlp/roberta_toxicity_classifier_v1'.

    Returns:
        _type_: _description_
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device)
    clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

def delete_toxic(df, 
                 threshold=0.5, 
                 size=20, 
                 ):
    sacrebleu_metric = load_metric("sacrebleu")
    rouge_metric = evaluate.load('rouge')
    ter_metric = evaluate.load("ter")
    print(df.head())
    toxic_sentences = df.reference[:size].tolist()
    toxic_sentences_list = [[t] for t in toxic_sentences]
    detoxified_text = []
    result = {
        "bleu": 0,
        "rouge1": 0,
        "rouge2": 0,
        "TER": 0
    }
    n = len(toxic_sentences)
    for i in trange(len(toxic_sentences)):
        words = toxic_sentences[i].split()
        toxic_scores = predict_toxicity(words)
        detoxified_sentence = " ".join([sentence for sentence, score in zip(words, toxic_scores) if score < threshold])
        detoxified_text.append(detoxified_sentence)

    result["bleu"] = sacrebleu_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)["score"]
    rouge_score = rouge_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)
    result["rouge1"] = rouge_score["rouge1"]
    result["rouge2"] = rouge_score["rouge2"]
    result["TER"] = ter_metric.compute(predictions=detoxified_text, references=toxic_sentences_list)["score"]
    # Let's print last 5 predictions
    print(result)
    return detoxified_text


In [4]:
delete_toxic(df)

0